# Install Package

In [1]:
!pip install skl2onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.4/298.4 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.1 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.5
    Uninstalling protobuf-4.25.5:
      Successfully uninstalled protobuf-4.25.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.62.3 requires protobuf>=4.21.6, but you have protobuf 3.20.2 which is incompatible.
tensorflow 2.17.1 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 3.20.2 which is incompatible.
tensorflow-metadata 1.13.1 requires protobuf<5,>=3.20.3, but you have protobuf 3.20.2 w

In [2]:
!pip install onnx

# Import Library

In [23]:
import numpy as np
import pickle
import joblib
import time
import os
import tensorflow as tf
from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import skl2onnx
import onnx

In [24]:
# Memuat dataset Iris
cancer = load_breast_cancer()
X = cancer.data
y = cancer.target

# Membagi data menjadi training dan testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Membuat dan melatih model RandomForest
model_rf = RandomForestClassifier(n_estimators=100, random_state=42)
model_rf.fit(X_train, y_train)

# Evaluasi model
y_pred = model_rf.predict(X_test)
print(f"Akurasi model RandomForest: {accuracy_score(y_test, y_pred)}")

Akurasi model RandomForest: 0.9649122807017544


# Pickle

In [25]:
# Menyimpan model dengan Pickle
start_time = time.time()
with open('model_rf_pickle.pkl', 'wb') as f:
    pickle.dump(model_rf, f)
pickle_time = time.time() - start_time

# Memuat model dengan Pickle
start_time = time.time()
with open('model_rf_pickle.pkl', 'rb') as f:
    model_rf_pickle = pickle.load(f)
pickle_load_time = time.time() - start_time

# Joblib

In [26]:
# Menyimpan model dengan Joblib
start_time = time.time()
joblib.dump(model_rf, 'model_rf_joblib.pkl')
joblib_time = time.time() - start_time

# Memuat model dengan Joblib
start_time = time.time()
model_rf_joblib = joblib.load('model_rf_joblib.pkl')
joblib_load_time = time.time() - start_time

# ONNX

In [27]:
# Konversi model RandomForest ke ONNX
start_time = time.time()
initial_types = [('input', skl2onnx.common.data_types.FloatTensorType([None, X_train.shape[1]]))]
onnx_model = skl2onnx.convert_sklearn(model_rf, initial_types=initial_types)

# Simpan model ONNX
with open('model_rf.onnx', 'wb') as f:
    f.write(onnx_model.SerializeToString())
onnx_time = time.time() - start_time

# Memuat model ONNX
start_time = time.time()
onnx_model_loaded = onnx.load('model_rf.onnx')
onnx_load_time = time.time() - start_time

# Tensorflow

In [28]:
# Membuat model TensorFlow untuk eksperimen ini
model_tf = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

# Kompilasi dan latih model TensorFlow
model_tf.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model_tf.fit(X_train, y_train, epochs=10, verbose=0)

# Menyimpan model TensorFlow
start_time = time.time()
model_tf.save('model_tf.h5')
tf_time = time.time() - start_time

# Memuat model TensorFlow
start_time = time.time()
model_tf_loaded = tf.keras.models.load_model('model_tf.h5')
tf_load_time = time.time() - start_time

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


# Model Analysis

In [29]:
# Mengukur ukuran file dari masing-masing metode serialisasi
pickle_size = os.path.getsize('model_rf_pickle.pkl')
joblib_size = os.path.getsize('model_rf_joblib.pkl')
onnx_size = os.path.getsize('model_rf.onnx')
tf_size = os.path.getsize('model_tf.h5')

# Menampilkan hasil
print(f"Pickle: Save time = {pickle_time:.4f}s, Load time = {pickle_load_time:.4f}s, File size = {pickle_size / 1024:.2f} KB")
print(f"Joblib: Save time = {joblib_time:.4f}s, Load time = {joblib_load_time:.4f}s, File size = {joblib_size / 1024:.2f} KB")
print(f"ONNX: Save time = {onnx_time:.4f}s, Load time = {onnx_load_time:.4f}s, File size = {onnx_size / 1024:.2f} KB")
print(f"TensorFlow: Save time = {tf_time:.4f}s, Load time = {tf_load_time:.4f}s, File size = {tf_size / 1024:.2f} KB")

Pickle: Save time = 0.0288s, Load time = 0.0096s, File size = 309.79 KB
Joblib: Save time = 0.0512s, Load time = 0.0300s, File size = 317.68 KB
ONNX: Save time = 0.0904s, Load time = 0.0035s, File size = 128.18 KB
TensorFlow: Save time = 0.0176s, Load time = 0.0345s, File size = 61.34 KB
